In [ ]:
%%bash

# Test Run with 10 designs 
target="PDL1.json"
target_name="${target%.json}"  # Strip the .json extension
target_path="/workspace/settings_target/$target"
log_dir="/workspace/outputs/$target_name"
log_file="$log_dir/${target_name}-bindcraft_log.txt"

# Ensure log directory exists
mkdir -p "$log_dir"

echo "[INFO] Checking target settings file..."
if [ ! -f "$target_path" ]; then
  echo "[ERROR] Target file '$target' not found in /workspace/settings_target/"
  exit 1
fi

echo "[INFO] Starting BindCraft with target '$target'"
echo "[INFO] Logging output to $log_file"

nohup python /app/bindcraft/bindcraft.py \
  --settings "$target_path" \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > "$log_file" 2>&1 &

echo "[INFO] BindCraft is now running in the background (PID=$!)"

In [ ]:
!tail -n 50 -f /workspace/outputs/${log_file}

In [1]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload(accept='.json', multiple=False)
display(uploader)

FileUpload(value=(), accept='.json', description='Upload')

In [25]:
import json
import os
from functools import partial

def parse_target_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def save_target_json(target_folder, filename, json_path, data):
    target_filepath = os.path.join(os.path.dirname(os.path.dirname(json_path)), target_folder)
    os.makedirs(target_filepath, exist_ok=True)
    target_filepath = os.path.join(target_filepath, filename)
    
    with open(f'{target_filepath}', 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {target_filepath}")


def target_editor_widget(data):
    widget_dict = {}
    for k, v in data.items():
         
        if isinstance(v, list):
            widget_dict[k] = widgets.Text(value=str(v), description=f'{k}:',
                                          style={'description_width': 'initial'})
        elif isinstance(v, int):
            widget_dict[k] = widgets.IntText(value=v, description=f'{k}:',
                                             style={'description_width': 'initial'})

        else:
            widget_dict[k] = widgets.Text(value=v, description=f'{k}:',
                                          style={'description_width': 'initial'})
        
    return widget_dict

def update_data(data, widget_dict):
    updated_data = {}
    for key, widget in widget_dict.items():
        w = widget.value
        #Convert back to a list if the original data was a list
        if isinstance(data[key], list):
            try:
                w = json.loads(w)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for key '{key}'. Keeping original value.")
                w = data[key]
        updated_data[key] = w
    return updated_data     

def on_save_clicked(button, data, widget_dict, target_folder, filename, json_path):
    updated_data = update_data(data, widget_dict)
    save_target_json(target_folder, filename, json_path, updated_data)
    
# Display the widgets and save button
json_path = '/home/yarrow/projects/bindcraft-runpod/workspace/settings_target/PDL1.json'
target_folder = 'test'
filename_widget = widgets.Text(value=os.path.basename(json_path),
                               description='File Name:',
                               layout=widgets.Layout(width='50%')
                               )

data = parse_target_json(json_path)
widget_dict = target_editor_widget(data)
save_button = widgets.Button(description="Save Changes", save_button_style='success')
save_button.on_click(partial(on_save_clicked,
                             data=data,
                             widget_dict=widget_dict,
                             target_folder=target_folder,
                             filename=filename_widget.value,
                             json_path=json_path))



display(filename_widget)
for widget in widget_dict.values():
    display(widget)

display(save_button)
widget_dict = {}

Text(value='PDL1.json', description='File Name:', layout=Layout(width='50%'))

Text(value='/workspace/outputs/PDL1', description='design_path:', style=TextStyle(description_width='initial')…

Text(value='PDL1-Binder', description='binder_name:', style=TextStyle(description_width='initial'))

Text(value='/workspace/inputs/PDL1.pdb', description='starting_pdb:', style=TextStyle(description_width='initi…

Text(value='A', description='chains:', style=TextStyle(description_width='initial'))

Text(value='56', description='target_hotspot_residues:', style=TextStyle(description_width='initial'))

Text(value='[65, 150]', description='lengths:', style=TextStyle(description_width='initial'))

IntText(value=10, description='number_of_final_designs:', style=DescriptionStyle(description_width='initial'))

Button(description='Save Changes', style=ButtonStyle())

In [ ]:
import json
import os
from pathlib import Path
from functools import partial
import ipywidgets as widgets
from IPython.display import display

def parse_target_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def save_target_json(target_folder, filename, json_target_path, data):
    base_dir = os.path.dirname(os.path.dirname(json_target_path))
    target_filepath = os.path.join(base_dir, target_folder)
    os.makedirs(target_filepath, exist_ok=True)
    
    full_path = os.path.join(target_filepath, filename)
    print(f"[DEBUG] Writing file to: {full_path}")

    with open(full_path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f'Data saved to {full_path}')

def update_data(data, widget_dict):
    updated_data = {}
    for key, widget in widget_dict.items():
        w = widget.value
        #Convert back to a list if the original data was a list
        if isinstance(data[key], list):
            try:
                w = json.loads(w)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for key '{key}'. Keeping original value.")
                w = data[key]
        updated_data[key] = w
    return updated_data     

def on_save_clicked(button, data, widget_dict, target_folder_widget, filename_widget, json_target_path):
    print("[DEBUG] Save button clicked")
    target_folder = target_folder_widget.value.strip()
    filename = filename_widget.value.strip()
    #if not target_folder or not filename:
    #    print (f'Please enter target folder and filename')
    #    return

    updated_data = update_data(data, widget_dict)
    save_target_json(target_folder, filename, json_target_path, updated_data)
    print(f'Saved parameters to {filename} in folder {target_folder}')


#UI Builder

def target_editor_widget(data):
    widget_dict = {}
    for k, v in data.items():
         
        if isinstance(v, list):
            widget_dict[k] = widgets.Text(value=str(v), description=f'{k}:',
                                          style={'description_width': 'initial'})
    
        elif isinstance(v, int):
            widget_dict[k] = widgets.IntText(value=v, description=f'{k}:',
                                             style={'description_width': 'initial'})

        else:
            widget_dict[k] = widgets.Text(value=v, description=f'{k}:',
                                          style={'description_width': 'initial'})
        
    return widget_dict

def main_launch_editor(json_target_path='/workspace/settings_target/PDL1.json'):
    data = parse_target_json(json_target_path)

    target_folder_widget = widgets.Text(value=Path(json_target_path).stem,
                                    description='Folder Name to save file to',
                                    style={'description_width': 'initial'}
                                   )
    filename_widget = widgets.Text(value=os.path.basename(json_target_path),
                               description='File Name:',
                               layout=widgets.Layout(width='50%')
                               )
    #Editable Field widgets
    widget_dict = target_editor_widget(data)

    save_button = widgets.Button(description="Save Changes", button_style='success')
    save_button.on_click(partial(on_save_clicked,
                             data=data,
                             widget_dict=widget_dict,
                             target_folder_widget=target_folder_widget,
                             filename_widget=filename_widget,
                             json_target_path=json_target_path
    ))
    #Display widgets
    display(filename_widget)
    display(target_folder_widget)
    for widget in widget_dict.values():
        display(widget)
    display(save_button)


In [ ]:
main_launch_editor('/home/yarrow/projects/bindcraft-runpod/settings_target/PDL1.json')

Text(value='PDL1.json', description='File Name:', layout=Layout(width='50%'))

Text(value='PDL1', description='Folder Name to save file to', style=TextStyle(description_width='initial'))

Text(value='/workspace/outputs/PDL1', description='design_path:', style=TextStyle(description_width='initial')…

Text(value='PDL1-Binder', description='binder_name:', style=TextStyle(description_width='initial'))

Text(value='/workspace/inputs/PDL1.pdb', description='starting_pdb:', style=TextStyle(description_width='initi…

...

Text(value='56', description='target_hotspot_residues:', style=TextStyle(description_width='initial'))

Text(value='[65, 150]', description='lengths:', style=TextStyle(description_width='initial'))

IntText(value=10, description='number_of_final_designs:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Save Changes', style=ButtonStyle())

TypeError: on_save_clicked() got an unexpected keyword argument 'filename'